In [ ]:
# only for cuda enabled laptop and desktop
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.utils.get_file('horse-or-human.zip','https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip')

In [ ]:
import os #operating system make available os function
import zipfile # zip and unzip

local_zip = '/home/aarav/.keras/datasets/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('horse-or-human')
zip_ref.close()

In [ ]:
# Directory with our training horse pictures
import os
train_horse_dir = os.path.join('horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('horse-or-human/humans')

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [ ]:
image_1 = load_img('horse-or-human/horses/horse01-7.png')
image_1

In [ ]:
import matplotlib.image as mpimg
import numpy as np
imd = mpimg.imread('horse-or-human/horses/horse01-7.png')
imd.shape

In [ ]:
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
folder_count = len(glob('horse-or-human/*'))
print(folder_count)

In [ ]:
IMG_HIEGHT = 300
IMG_WIDHT = 300

In [1]:
image_gen_train = ImageDataGenerator(rescale=1./255,
                                     rotation_range=25,
                                     zoom_range=0.1,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     shear_range=0.2,
                                     horizontal_flip=True,
                                    validation_split=0.2,)

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(directory='horse-or-human',
                                                    shuffle=True,
                                                    target_size=(IMG_HIEGHT,IMG_HIEGHT),
                                                    subset='training',
                                                    class_mode='binary')

In [ ]:
train_data_gen.class_indices

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255,
                                  validation_split=0.2)
val_data_gen = image_gen_val.flow_from_directory(directory='horse-or-human',
                                                target_size=(IMG_HIEGHT,IMG_WIDHT),
                                                subset='validation',
                                                class_mode='binary')

In [ ]:
import time

In [ ]:
sample_data, label = next(train_data_gen)

In [ ]:
sample_data.shape

In [ ]:
label

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(sample_data[i], cmap=plt.cm.binary)
    plt.xlabel(label[i])
plt.show()

In [ ]:
from keras import layers
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (300, 300, 3),include_top = False, weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.BatchNormalization()(x)

x = layers.Dense(32, activation='relu')(x)

x = layers.Dense(1, activation='sigmoid')(x)

vgg_model = tf.keras.models.Model(base_model.input, x)

In [ ]:
vgg_model.summary()

In [ ]:
vgg_model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
tf.keras.utils.plot_model(vgg_model,show_shapes=True)

In [ ]:
t1 = time.time()
history = vgg_model.fit_generator(train_data_gen, epochs=10, validation_data=val_data_gen)
t2 = time.time()

In [ ]:
print('Time to train model:',round((t2-t1)/60, 2), 'minuts')

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')
plt.figure(figsize=(12,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(300, 300 ,3), include_top=False, weights="imagenet")

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.BatchNormalization()(x)

x = layers.Dense(32, activation='relu')(x)
#x = layers.Dense(16, activation='relu')(x)

x = layers.Dense(1, activation='sigmoid')(x)

res_net_model = tf.keras.models.Model(base_model.input, x)


res_net_model.summary()

In [ ]:
res_net_model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
t1 = time.time()
history = res_net_model.fit_generator(train_data_gen, epochs=2, validation_data=val_data_gen)
t2 = time.time()

In [ ]:
print('Time to train model:',round((t2-t1)/60, 2), 'minuts')
train_score = res_net_model.evaluate_generator(train_data_gen)
valid_score = res_net_model.evaluate_generator(val_data_gen)
print("Train_Accuracy = ", train_score[1])
print("Validation_Accuracy = ", valid_score[1])

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = (300, 300, 3), include_top = False, weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.BatchNormalization()(x)

x = layers.Dense(32, activation='relu')(x)

x = layers.Dense(1, activation='sigmoid')(x)

inception_model = tf.keras.models.Model(base_model.input, x)

In [ ]:
inception_model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
t1 = time.time()
history = inception_model.fit_generator(train_data_gen, epochs=2, validation_data=val_data_gen)
t2 = time.time()
print('Time to train model:',round((t2-t1)/60, 2), 'minuts')

In [ ]:
#Efficientnet#

In [ ]:
!pip install -U efficientnet

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0

base_model = EfficientNetB0(input_shape = (300, 300, 3), include_top = False, weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.BatchNormalization()(x)

x = layers.Dense(32, activation='relu')(x)

x = layers.Dense(1, activation='sigmoid')(x)

efficientnet = tf.keras.models.Model(base_model.input, x)

In [ ]:
efficientnet.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
t1 = time.time()
history = efficientnet.fit_generator(train_data_gen, epochs=2, validation_data=val_data_gen)
t2 = time.time()